##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text classification with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_classification_rnn"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_classification_rnn.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_classification_rnn.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_classification_rnn.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This text classification tutorial trains a [recurrent neural network](https://developers.google.com/machine-learning/glossary/#recurrent_neural_network) on the [IMDB large movie review dataset](http://ai.stanford.edu/~amaas/data/sentiment/) for sentiment analysis.

## Setup

In [ ]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf


Import `matplotlib` and create a helper function to plot graphs:

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

## Setup input pipeline


The IMDB large movie review dataset is a *binary classification* dataset—all the reviews have either a *positive* or *negative* sentiment.

Download the dataset using [TFDS](https://www.tensorflow.org/datasets). See the [loading text tutorial](https://www.tensorflow.org/tutorials/load_data/text) for details on how to load this sort of data manually.


In [ ]:
import pandas as pd
df_gender = pd.read_csv("gender.csv",encoding='latin1')
df_age = pd.read_csv("age.csv",encoding='latin1')

Initially this returns a dataset of (text, label pairs):

Next shuffle the data for training and create batches of these `(text, label)` pairs:

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df_gender['Content'],df_gender['gender'], train_size=0.9, stratify=df_gender['gender'])

In [ ]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])


In [ ]:
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_6 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
len(X_train)


8550

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=4)

model.fit(X_train, y_train, epochs=35,callbacks=[callback])


Epoch 1/35
268/268 [==============================] - 93s 348ms/step - loss: 0.6147 - accuracy: 0.6636 - precision: 0.6710 - recall: 0.8186
Epoch 2/35
268/268 [==============================] - 94s 350ms/step - loss: 0.6144 - accuracy: 0.6574 - precision: 0.6717 - recall: 0.7951
Epoch 3/35
268/268 [==============================] - 93s 348ms/step - loss: 0.6037 - accuracy: 0.6806 - precision: 0.6893 - recall: 0.8130
Epoch 4/35
268/268 [==============================] - 93s 347ms/step - loss: 0.5969 - accuracy: 0.6759 - precision: 0.6851 - recall: 0.8116
Epoch 5/35
268/268 [==============================] - 94s 351ms/step - loss: 0.5957 - accuracy: 0.6798 - precision: 0.6947 - recall: 0.7943
Epoch 6/35
268/268 [==============================] - 93s 347ms/step - loss: 0.5901 - accuracy: 0.6807 - precision: 0.6933 - recall: 0.8012
Epoch 7/35
268/268 [==============================] - 94s 349ms/step - loss: 0.5911 - accuracy: 0.6792 - precision: 0.6916 - recall: 0.8016
Epoch 8/35
268/268 [

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive
tf.saved_model.save(model, 'models/bert_v1')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive


In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

30/30 [==============================] - 11s 343ms/step


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.69      0.55      0.61       402
           1       0.71      0.82      0.76       548

    accuracy                           0.71       950
   macro avg       0.70      0.68      0.69       950
weighted avg       0.70      0.71      0.70       950



In [ ]:
from official.nlp import optimization  # to create AdamW optimizer

epochs = 5
steps_per_epoch = len(df_gender)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
classifier_model.fit(X_train, y_train, epochs=10)


Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/10
223/223 [==============================] - 67s 241ms/step - loss: 0.6833 - binary_accuracy: 0.5120
Epoch 2/10
223/223 [==============================] - 52s 235ms/step - loss: 0.6753 - binary_accuracy: 0.5222
Epoch 3/10
223/223 [==============================] - 53s 239ms/step - loss: 0.6764 - binary_accuracy: 0.5272
Epoch 4/10
223/223 [==============================] - 53s 238ms/step - loss: 0.6780 - binary_accuracy: 0.5231
Epoch 5/10
223/223 [==============================] - 56s 250ms/step - loss: 0.6793 - binary_accuracy: 0.5229
Epoch 6/10
135/223 [=================>............] - ETA: 20s - loss: 0.6773 - binary_accuracy: 0.5278

KeyboardInterrupt: ignored